In [16]:
import pandas as pd
import numpy as np
import re
import unicodedata


In [2]:
t1 = pd.read_csv("nacimiento_edad_semana.csv", sep=";", encoding="latin1") #OK1
t2 = pd.read_csv("partos_nacimientos.csv", sep=";", encoding="latin1") 
t3 = pd.read_csv("porcentaje_nacidos_nacionalidad.csv", sep=";", encoding="latin1") #OK2
df_fecundidad = pd.read_csv("tasa_fecundidad_nacionalidad.csv", sep=";", encoding="latin1") #me gusta mucho las columnas
t5 = pd.read_csv("aborto_metodo.csv", sep=";", encoding="latin1")#OK3
t6 = pd.read_csv("aborto_edad.csv", sep=";", encoding="latin1")#OK3
t7 = pd.read_csv("aborto_motivo.csv", sep=";", encoding="latin1")#OK3
t8 = pd.read_csv("aborto_gestacion.csv", sep=";", encoding="latin1")#OK3
t9 = pd.read_csv("conyuntural_fecundidad.csv", sep=";", encoding="latin1") #OK2
t10 = pd.read_csv("nacidos_semana_gestacion.csv", sep=";", encoding="latin1") #OK1
t11 = pd.read_csv("hombres_embarazo.csv", sep=";", encoding="latin1")
df_ive_cat = pd.read_csv("ive_catalunya.csv", sep=";", encoding="latin1")
df_mujeres = pd.read_csv("mujeres_intencion_3años.csv", sep=";", encoding="latin1")


Antes de juntar las tablas he mirado que todas las columnas coincidan y que si se tiene que modificar el nombre de las columnas o agrupar se pueda para que no se pierda información.

In [3]:
def convertir_edad_a_grupo(valor):
    # Casos directos
    if valor == "Todas las edades":
        return "Todas las edades"
    if valor == "Menos de 15 años":
        return "Menos de 15 años"
    if valor == "50 y más años":
        return "50 y más años"
    
    # Intentar extraer número de edad
    match = re.search(r'\d+', str(valor))
    if not match:
        return None  # o valor original si prefieres
    
    edad = int(match.group())
    
    if 15 <= edad <= 19:
        return "De 15 a 19 años"
    elif 20 <= edad <= 24:
        return "De 20 a 24 años"
    elif 25 <= edad <= 29:
        return "De 25 a 29 años"
    elif 30 <= edad <= 34:
        return "De 30 a 34 años"
    elif 35 <= edad <= 39:
        return "De 35 a 39 años"
    elif 40 <= edad <= 44:
        return "De 40 a 44 años"
    elif 45 <= edad <= 49:
        return "De 45 a 49 años"
    elif edad >= 50:
        return "50 y más años"
    elif edad < 15:
        return "Menos de 15 años"
    
    return None

t1["Edad de la madre"] = t1["Edad de la madre"].apply(convertir_edad_a_grupo)
t10["Edad de la madre"] = t10["Edad de la madre"].apply(convertir_edad_a_grupo)

t1 = t1.rename(columns={"Edad de la madre": "Grupo de edad de la madre"})
t10 = t10.rename(columns={"Edad de la madre": "Grupo de edad de la madre"})



In [4]:
t3 = t3.drop(columns=["Totales Territoriales"])
t9 = t9.drop(columns=["Totales Territoriales"])

In [5]:
t10 = t10.rename(columns={"Nacional y provincias": "Provincias"})

Agrupamos por categoria 
1. Nacionalidad
2. Fecundidad
3. Aborto
4. Catalunya IVE

In [6]:
#df_nacimientos = pd.concat([t1, t10], ignore_index=True)
df_nacimientos_nacionalidad = pd.concat([t3, t9], ignore_index=True)
df_abortos = pd.concat([t5, t6, t7, t8], ignore_index=True)
df_ive_cat
df_mujeres
df_fecundidad

,Comunidades y Ciudades AutÃ³nomas,Nacionalidad,Edad,Periodo,Total
0,Total Nacional,Ambas nacionalidades,De 10 a 14 aÃ±os,2024,"0,07"
1,Total Nacional,Ambas nacionalidades,De 10 a 14 aÃ±os,2023,"0,06"
2,Total Nacional,Ambas nacionalidades,De 10 a 14 aÃ±os,2022,"0,06"
3,Total Nacional,Ambas nacionalidades,De 10 a 14 aÃ±os,2021,"0,05"
4,Total Nacional,Ambas nacionalidades,De 10 a 14 aÃ±os,2020,"0,05"
...,...,...,...,...,...
26995,19 Melilla,Extranjera,50 y mÃ¡s aÃ±os,1979,NaN
26996,19 Melilla,Extranjera,50 y mÃ¡s aÃ±os,1978,NaN
26997,19 Melilla,Extranjera,50 y mÃ¡s aÃ±os,1977,NaN
26998,19 Melilla,Extranjera,50 y mÃ¡s aÃ±os,1976,NaN


## Limpieza del dataframe df_abortos

In [7]:
#Tipo territorio	CÃ³digo territorio	Territorio Unidad	Estado dato
df_abortos = df_abortos.drop(columns=[
    "Tipo territorio",
    "CÃ³digo territorio",
    "Territorio Unidad",
    "Estado dato"
], errors="ignore")

In [8]:
df_abortos["Concepto"] = (df_abortos['Concepto']
.str.replace('Abortos voluntarios de mujeres residentes realizados por el método de ', '', regex=False)
.str.replace('Abortos voluntarios de mujeres residentes realizados por ', '', regex=False)
.str.replace(' / ', ' ', regex=False)
.str.strip()
)

In [15]:
#df_abortos = df_abortos.drop(columns=["Territorio", "Unidad"])
df_abortos.head(5)

,AÃ±o,Concepto,Valor
0,2011,el mÃ©todo de dilataciÃ³n y evacuaciÃ³n,3728
1,2011,el mÃ©todo de mifepristona,424
2,2011,el mÃ©todo de dilataciÃ³n y aspiraciÃ³n,16231
3,2011,el mÃ©todo de prostaglandinas,304
4,2011,otros mÃ©todos no consta,76


In [12]:
df_abortos.head()
#nose si fue buena idea haber juntado las tablas porque uno habla del metodo y el otro el de la edad
#añadir una columna para el origen de cada tabla.
#pivotar y despivotar mirarlos

#df_abortos["Concepto"].unique()

,AÃ±o,Concepto,Territorio,Valor,Unidad
0,2011,el mÃ©todo de dilataciÃ³n y evacuaciÃ³n,NaN,3728,NÃºmero
1,2011,el mÃ©todo de mifepristona,NaN,424,NÃºmero
2,2011,el mÃ©todo de dilataciÃ³n y aspiraciÃ³n,NaN,16231,NÃºmero
3,2011,el mÃ©todo de prostaglandinas,NaN,304,NÃºmero
4,2011,otros mÃ©todos no consta,NaN,76,NÃºmero


In [11]:
df_abortos_long = pd.melt(
    df_abortos,
    id_vars=["Año", "Concepto"],
    var_name="Variable",
    value_name="Valor"
)
df_abortos_long

ValueError: value_name (Valor) cannot match an element in the DataFrame columns.